Mahounan Y.

November 3rd, 2021

## Intro
In this file, I am practicing the basic commands to link `python` to `SQLite` using the **sqlite3** library. Three steps are needed:

`1. Import the library`

`2. Create a connection between python and SQLite`

`3. Create a cursor that will be used to move back and forth between python and sqlite`

In [21]:
#Import the library
import sqlite3

#Create connection to database
connection = sqlite3.connect("first.db")

#Create cursor object
cursor = connection.cursor()

In [22]:
# Create students table
cursor.execute('''CREATE TABLE students (
                    id INTEGER PRIMARY KEY,
                    name TEXT NOT NULL,
                    email TEXT NOT NULL UNIQUE,
                    major_code INTEGER,
                    grad_date datetime,
                    grade REAL NOT NULL)''')

OperationalError: table students already exists

In [23]:
# Add a row of data to students table
cursor.execute('''INSERT INTO students VALUES (101, 'Alex', 'alex@codeu.com', 32, '2022-05-16', 'Pass')''')

IntegrityError: UNIQUE constraint failed: students.id

In [4]:
# Insert multiple values into table at once
students = [(102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass'),
            (103, 'Stacy', 'stacy@codeu.com', 10, '2022-05-16', 'Pass'),
            (104, 'Angela', 'angela@codeu.com', 21, '2022-12-20', 'Pass'),
            (105, 'Mark', 'mark@codeu.com', 21, '2022-12-20', 'Fail'),
            (106, 'Nathan', 'nathaniel@codeu.com', 21, '2022-12-20', 'Pass')
            ]

In [5]:
# Insert values into the students table
cursor.executemany('''INSERT INTO students VALUES (?,?,?,?,?,?)''', students)

We use 6 question marks as placeholders to represent each of the 6 fields in the database that we will insert values into. Having made changes to our database, we need to ensure that the changes will be visible to others who may be working with our database. We use the commit() method on our connection to make sure these changes are committed.

In [24]:
# Commit changes to database
connection.commit()

In [25]:
# Iterate through all rows in students table
for row in cursor.execute("SELECT * FROM students"):
    print(row)

(101, 'Alex', 'alex@codeu.com', 32, '2022-05-16', 'Pass')
(102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass')
(103, 'Stacy', 'stacy@codeu.com', 10, '2022-05-16', 'Pass')
(104, 'Angela', 'angela@codeu.com', 21, '2022-12-20', 'Pass')
(105, 'Mark', 'mark@codeu.com', 21, '2022-12-20', 'Fail')
(106, 'Nathan', 'nathaniel@codeu.com', 21, '2022-12-20', 'Pass')


There are a number of sqlite3 methods that will retrieve data, these being

 - fetchone()
 - fetchmany()
 - fetchall()

In [26]:
# Return first row in students
cursor.execute("SELECT * FROM students").fetchone()


(101, 'Alex', 'alex@codeu.com', 32, '2022-05-16', 'Pass')

In [27]:
# Return first three rows in students
cursor.execute("SELECT * FROM students").fetchmany(3)

[(101, 'Alex', 'alex@codeu.com', 32, '2022-05-16', 'Pass'),
 (102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass'),
 (103, 'Stacy', 'stacy@codeu.com', 10, '2022-05-16', 'Pass')]

In [28]:
# Return all rows in students
cursor.execute("SELECT * FROM students").fetchall()

[(101, 'Alex', 'alex@codeu.com', 32, '2022-05-16', 'Pass'),
 (102, 'Joe', 'joseph@codeu.com', 32, '2022-05-16', 'Pass'),
 (103, 'Stacy', 'stacy@codeu.com', 10, '2022-05-16', 'Pass'),
 (104, 'Angela', 'angela@codeu.com', 21, '2022-12-20', 'Pass'),
 (105, 'Mark', 'mark@codeu.com', 21, '2022-12-20', 'Fail'),
 (106, 'Nathan', 'nathaniel@codeu.com', 21, '2022-12-20', 'Pass')]

All of the SQLite syntax will work in Python. I can use clauses like `WHERE`, `COUNT`, etc.

In [29]:
# Return the number of rows with a passing grade
cursor.execute("""SELECT COUNT(*) FROM students WHERE Grade = 'Pass';""").fetchone()

(5,)

Let’s say that we would like to find the average of the major codes field. We can use Python methods `sum()` and `len()` on our result set to obtain the mean value of the field.

In [30]:
# Create a list of tuples of the major codes
major_codes = cursor.execute("SELECT major_code FROM students;").fetchall()
 
# Obtain the average of the tuple list by using for loops
sum = 0
for num in major_codes: 
    for i in num: 
        sum = sum + i 
average = sum / len(major_codes)
 
# Show average
print(average)

22.833333333333332


## SQLite with Pandas

Let’s use Pandas to transform our SQLite database into a Pandas DataFrame. First we will import Pandas into Python, then call the Pandas `read_sql_query()` method, that takes in a query and a connection as parameters and returns a DataFrame corresponding to the output of the query:

In [32]:
import pandas as pd

# Create a new dataframe from the result set
df = pd.read_sql_query('''SELECT * from students;''', connection)
 
# Show new dataframe
print(df)

    id    name                email  major_code   grad_date grade
0  101    Alex       alex@codeu.com          32  2022-05-16  Pass
1  102     Joe     joseph@codeu.com          32  2022-05-16  Pass
2  103   Stacy      stacy@codeu.com          10  2022-05-16  Pass
3  104  Angela     angela@codeu.com          21  2022-12-20  Pass
4  105    Mark       mark@codeu.com          21  2022-12-20  Fail
5  106  Nathan  nathaniel@codeu.com          21  2022-12-20  Pass


We can create a number of DataFrames based on any query. For instance, if we want to create a DataFrame containing only those rows where the major code was equal to 21, we can use the `WHERE` clause within the `read_sql_query()` method:

In [33]:
# Create a new dataframe from the result set
df_major21 = pd.read_sql_query('''SELECT * from students WHERE major_code = 21;''', connection)
 
# Show new dataframe
print(df_major21)

    id    name                email  major_code   grad_date grade
0  104  Angela     angela@codeu.com          21  2022-12-20  Pass
1  105    Mark       mark@codeu.com          21  2022-12-20  Fail
2  106  Nathan  nathaniel@codeu.com          21  2022-12-20  Pass


Now that we are working with Pandas, if we wanted to find the average of the major codes, we could simply use the Pandas `mean()` method on the major_code field:

In [34]:
# Return the average of major code
df['major_code'].mean()

22.833333333333332